In [10]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import string
import tweepy
import re

from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tweepy import OAuthHandler


def get_tokens():
   with open('FirstContactWithTensorFlow.txt', 'r') as tf:
    text = tf.read()
    lowers = text.lower()
    #tokens = nltk.word_tokenize(text)
    puntuacióntext_no_punctuation = lowers.translate(str.maketrans('','',string.punctuation))

    #lowers = text.lower()
    #no_punctuation = lowers.translate(None, string.punctuation)
    tokens = nltk.word_tokenize(puntuacióntext_no_punctuation)
    return tokens


##### Task 2.6 ######

tokens = get_tokens()
#count = Counter(tokens)
#print(count)
filtered = [w for w in tokens if not w in stopwords.words('english')]
count = Counter(filtered)
print(len(count))


###### Task 2.7 #######

# Nos conectamos con la API de tweepy
consumer_key = 'X5EqKwcj2QSpcSlsqoBi29eCb'
consumer_secret = '6i9NqIFCGOrhZzUTlRe8lPTzevyEj8eA8lm8eWwF9MMrabHanq'
access_token = '982692299648925696-MlIqFRw5K8vNOmoGWD4bMoX0dpGo98n'
access_secret = 'at2hhW8W6wGz3mmgxJPCJSnu0UC6wku4l9iJ4U5CzxTdI'
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret) 
api = tweepy.API(auth)
user = api.me()

##### Task 2.9 #####
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE) 

# Ejemplo de tokenisation
tweet = 'RT @JordiTorresBCN: just an example! :D http://JordiTorres.Barcelona #masterMEI'
print(preprocess(tweet))

def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2994
['RT', '@JordiTorresBCN', ':', 'just', 'an', 'example', '!', ':D', 'http://JordiTorres.Barcelona', '#masterMEI']
